In [1]:
#libraries
import cv2
import numpy as np
import types
import tkinter as tk
from tkinter import *
from tkinter.filedialog import *
from PIL import ImageTk,Image
from stegano import exifHeader as stg
from tkinter import messagebox
import zlib
import struct
from typing import BinaryIO
from typing import Tuple
import os

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def messageToBinary(message):
    if type(message) == str:
        return ''.join([ format(ord(i), "08b") for i in message ])
    elif type(message) == bytes or type(message) == np.ndarray:
        return [ format(i, "08b") for i in message ]
    elif type(message) == int or type(message) == np.uint8:
        return format(message, "08b")
    else:
        raise TypeError("Input type not supported")

In [4]:
#Function that encodes one image into another - better hiding or better quality
def hideData(image, image2, n):
  # check if encoding is possible
    n_bytes = image.shape[0] * image.shape[1] * 3
    h_bytes = image2.shape[0] * image2.shape[1] * 3
  # if it's not => error
    if h_bytes > n_bytes:
        raise ValueError("Error encountered: insufficient bytes, need bigger cover image or smaller hidden one !!")
# we go through both images, pixel by pixel
    if (n==3): #if we want better hiding, we will only hide the first 3 MSB from each channel of each pixel from the
        #hidden image
        for v1,v2 in zip(image, image2):
            for p1, p2 in zip(v1,v2):
              # extract and convert to binary the values of each channel (Red, Green, Blue)
                r, g, b = messageToBinary(p1)
                rh, gh, bh = messageToBinary(p2)
                p1[0] = int(r[:(8-n)]+ rh[:n], 2) #R channel is made of the first 4 bits of the cover image and the last 4 bits of
                # the hidden image; identical procedure for both G and B channels
                p1[1] = int(g[:(8-n)] + gh[:n], 2)
                p1[2] = int(b[:(8-n)] + bh[:n], 2)
    elif (n==4): #if we want a better image quality when recovering the image, we will hide the first 4
        for v1,v2 in zip(image, image2):
            for p1, p2 in zip(v1,v2):
              # extract and convert to binary the values of each channel (Red, Green, Blue)
                r, g, b = messageToBinary(p1)
                rh, gh, bh = messageToBinary(p2)
                p1[0] = int(r[:n]+ rh[:n], 2) #R channel is made of the first 4 bits of the cover image and the last 4 bits of
                # the hidden image; identical procedure for both G and B channels
                p1[1] = int(g[:n] + gh[:n], 2)
                p1[2] = int(b[:n] + bh[:n], 2)

    return image

In [5]:
#function that generates a key to encode the message a little more
def Key_Gen(secret_message):
    n = len(secret_message)
    key = os.urandom(n)
    return key
def MessageEncodeKey(secret_message,keyfile):
    n = len(secret_message)
    key = os.urandom(n)
    with open(keyfile, 'wb') as f:
        f.write(key)
    return key

#function that uses the XOR operation between key and message so the message gets encrypted
def MEXOR(key, secret_message, n):
    if (n == 0):
        key = messageToBinary(key)
    else:
        key = [ key[i: i+8] for i in range(0, len(key), 8) ]
    secret_message = messageToBinary(secret_message)
    secret_message = [ secret_message[i: i+8] for i in range(0, len(secret_message), 8) ]
    xor_mess = []
    for key_val, mess_val in zip(key, secret_message):
        xor_mess.append(int(key_val,2) ^ int(mess_val,2))
    final_message = ""
    for val in xor_mess:
        val = messageToBinary(val)
        final_message += val
    return final_message

#function that hides a message inside an image
def hideDataMessage(image, secret_message, n, keyfile):

  # calculate maximum number of bytes that can be encoded
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8

  # check if the message is too long for encoding in this image
    if len(secret_message) > n_bytes:
        raise ValueError("ERROR: INSUFFICIENT NUMBER OF BYTES")
    if (n == 0):
        key = MessageEncodeKey(secret_message,keyfile)
    else:
        key = Key_Gen(secret_message)
    secret_message = MEXOR(key, secret_message, 0)
    terminator = "#"
    terminator = messageToBinary(terminator)
    secret_message += terminator # message terminator
    data_index = 0
    binary_secret_msg = secret_message
    data_len = len(binary_secret_msg) #Find the length of data that needs to be hidden
    for values in image:
        for pixel in values:
          # convert RGB values to binary format
            r, g, b = messageToBinary(pixel)
          # modify the least significant bit only if there is still data to store
            if data_index < data_len:
              # hide the data into least significant bit of red pixel
                pixel[0] = int(r[:-1] + binary_secret_msg[data_index], 2)
                data_index += 1
            if data_index < data_len:
              # hide the data into least significant bit of green pixel
                pixel[1] = int(g[:-1] + binary_secret_msg[data_index], 2)
                data_index += 1
            if data_index < data_len:
              # hide the data into least significant bit of blue pixel
                pixel[2] = int(b[:-1] + binary_secret_msg[data_index], 2)
                data_index += 1
          # if data is encoded, just break out of the loop
            if data_index >= data_len:
                break

    return image, key

In [6]:
#Function that restores the hidden image - better quality
def showData_quality(image):
    for values in image:
        for pixel in values:
            r, g, b = messageToBinary(pixel) #convert the red,green and blue values into binary format
            pixel[0] = int(r[4:] + "0000", 2);
            pixel[1] = int(g[4:] + "0000", 2);
            pixel[2] = int(b[4:] + "0000", 2);
    return image

#function that restores the hidden image - better hiding
def showData(image):
    for values in image:
        for pixel in values:
            r, g, b = messageToBinary(pixel) #convert the red,green and blue values into binary format
            pixel[0] = int(r[3:] + "00000", 2);
            pixel[1] = int(g[3:] + "00000", 2);
            pixel[2] = int(b[3:] + "00000", 2);
    return image

In [7]:
def strToInt(val):
    i = 7;
    result = 0;
    for each in val:
        if (each == "1"):
            result += (2**i)
        i -= 1;
    return result

def showDataMessage(image, path):

    binary_data = ""
    for values in image:
        for pixel in values:
            r, g, b = messageToBinary(pixel) #convert the red,green and blue values into binary format
            binary_data += r[-1] #extracting data from the least significant bit of red pixel
            binary_data += g[-1] #extracting data from the least significant bit of red pixel
            binary_data += b[-1] #extracting data from the least significant bit of red pixel
  # split by 8-bits
    all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
  # convert from bits to characters
    decoded_data = ""
    for byte in all_bytes:
        if chr(int(byte, 2)) == "#": # verific daca am ajuns la delimitator
            break
        decoded_data += chr(int(byte, 2))
    fd = os.open(path, os.O_RDONLY)
    n = os.path.getsize(path)
    key = os.read(fd, n)
    test = MEXOR(key, decoded_data, 0)
    hidden_message = [ test[i: i+8] for i in range(0, len(test), 8) ]
    for val in hidden_message:
        val = strToInt(val)
        val = chr(val)
    return hidden_message #remove the delimeter to show the original hidden message

def showDataMessage_key(image, key):

    binary_data = ""
    for values in image:
        for pixel in values:
            r, g, b = messageToBinary(pixel) #convert the red,green and blue values into binary format
            binary_data += r[-1] #extracting data from the least significant bit of red pixel
            binary_data += g[-1] #extracting data from the least significant bit of red pixel
            binary_data += b[-1] #extracting data from the least significant bit of red pixel
  # split by 8-bits
    all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
  # convert from bits to characters
    decoded_data = ""
    for byte in all_bytes:
        if chr(int(byte, 2)) == "#": # verific daca am ajuns la delimitator
            break
        decoded_data += chr(int(byte, 2))
    
    key = bytes(key, 'utf-8')
    test = MEXOR(key, decoded_data, 1)

    hidden_message = [ test[i: i+8] for i in range(0, len(test), 8) ]
    for val in hidden_message:
        val = strToInt(val)
        val = chr(val)
    return hidden_message #remove the delimeter to show the original hidden message

In [8]:
# Function that initiates the encoding of an image inside another
def encode_img(image_name, image_name2, filename, answer): 
    image = cv2.imread(image_name) # Read the input image using OpenCV-Python.
    length = image.shape[0]
    height = image.shape[1]
    image2 = cv2.imread(image_name2) # Read the input image using OpenCV-Python.
    if (length != image2.shape[0] or height != image2.shape[1]):
        image2 = cv2.resize(image2, (length, height)) #resize the image as per the requirement
    if (answer == "yes"):
        encoded_image = hideData(image, image2, 4)# call the hideData function to hide the secret message into the selected image
    elif (answer == "no"):
        encoded_image = hideData(image, image2, 3)
    else:
        print("ERROR: INVALID ANSWER")
    cv2.imwrite(filename, encoded_image)

In [9]:
def encode_text(image_name, data, filename, keyfile):
    image = cv2.imread(image_name) # Read the input image using OpenCV-Python.
  #It is a library of Python bindings designed to solve computer vision problems. 
    if (len(data) == 0): 
        raise ValueError('Data is empty')
    encoded_image, key = hideDataMessage(image, data, 0, keyfile) # call the hideData function to 
    #hide the secret message into the selected image 
    cv2.imwrite(filename, encoded_image)

def encode_text_key(image_name, data, filename):
    image = cv2.imread(image_name) # Read the input image using OpenCV-Python.
#It is a library of Python bindings designed to solve computer vision problems. 
    if (len(data) == 0): 
        raise ValueError('Data is empty')
    encoded_image, key = hideDataMessage(image, data, 1, "key.txt") # call the hideData function to hide the secret message into the selected image
    save_png(encoded_image, filename, key)

In [10]:
#==========================================================
#Below is a section of the code in which a PNG image is created
Pixel = Tuple[int, int, int]
HEADER = b'\x89PNG\r\n\x1A\n'
def get_checksum(chunk_type, data):
    checksum = zlib.crc32(chunk_type)
    checksum = zlib.crc32(data, checksum)
    return checksum

def chunk(out, chunk_type, data):
    out.write(struct.pack('>I', len(data)))
    out.write(chunk_type)
    out.write(data)

    checksum = get_checksum(chunk_type, data)
    out.write(struct.pack('>I', checksum))

def make_ihdr(width: int, height: int, bit_depth: int, color_type: int) -> bytes:
    return struct.pack('>2I5B', width, height, bit_depth, color_type, 0, 0, 0)

def encode_data(img):
    ret = []
    #swap pixel channels red and blue to switch to RGB which is used by Windows
    for values in img:
        for pval in values:
            aux = pval[0]
            pval[0] = pval[2]
            pval[2] = aux
    for row in img:
        ret.append(0)
        color_values = [
            color_value
            for pixel in row
            for color_value in pixel
        ]
        ret.extend(color_values)

    return ret

def compress_data(data):
    data_bytes = bytearray(data)
    return zlib.compress(data_bytes)

def make_idat(img):
    encoded_data = encode_data(img)
    compressed_data = compress_data(encoded_data)
    return compressed_data

def dump_png(out, img, key):
    out.write(HEADER)  # start by writing the header
    assert len(img) > 0  # assume we were not given empty image data
    width = len(img[0])
    height = len(img)
    bit_depth = 8  # bits per pixel
    color_type = 2  # pixel is RGB triple
    ihdr_data = make_ihdr(width, height, bit_depth, color_type)
    chunk(out, b'IHDR', ihdr_data)
    compressed_data = make_idat(img)
    chunk(out, b'IDAT', data=compressed_data)
    x = ""
    for val in key:
        x+=str(messageToBinary(val))
    x = bytes(x, 'utf-8')
    chunk(out, b'puNK', data=x)
    chunk(out, b'IEND', data=b'')

#function that initiates the writing of a png file
def save_png(img, filename, key):
    with open(filename, 'wb') as out:
        dump_png(out, img, key)

In [11]:
#======================================================
#Function that reads a png binary
def open_png(filename):
    fh = open(filename, 'rb')

    first_eight = [137,80,78,71,13,10,26,10]
    valid_count = 0;
    i = 0
    chunksize = 1
    while (i < 8):
        first_byte = fh.read(chunksize)
        val = struct.unpack('B',first_byte)[0]
        if first_eight[i] == val:
            valid_count += 1
        else:
            print( "ERROR: Invalid PNG signature")
            exit(-1)
        i += 1;

    if valid_count == 8:
        print( "Valid PNG signature")
    return fh

def process_ihdr(fh):
# The ! is necessary because PNGs being Portable "NETWORK" Graphics use
# network byte ordering (which is big-endian). 
# Unnecessary information will not be printed
    width = struct.unpack('!I', fh.read(4))[0]
    height = struct.unpack('!I', fh.read(4))[0]
    bit_depth = struct.unpack('B', fh.read(1))[0]
    colour_type = struct.unpack('B', fh.read(1))[0]
    compression_method = struct.unpack('B', fh.read(1))[0]
    filter_method = struct.unpack('B', fh.read(1))[0]
    interlace_method = struct.unpack('B', fh.read(1))[0]
    fh.read(4)
        
def process_punk(fh, chunk_length):
    width = fh.read(chunk_length)
    width = width.decode('utf-8')
    key = [ width[i: i+8] for i in range(0, len(width), 8) ]
    return width

def process_chunks(filename):
    fh = open_png(filename)
    idats_found = 0
    idats_bytes = 0
    while (1):
        chlen = fh.read(4)
        if (len(chlen) == 0):
            break

        chunk_length = struct.unpack('!I',chlen)[0]

        chunk_type = ""
        for i in range(4):
            s=struct.unpack('c',fh.read(1))[0]
            chunk_type += (s.decode('UTF-8'))
# If needed, information about chunks can be printed.
# Processing of the chunks is mandatory in order to get to the needed information
        if (chunk_type == "IHDR"):
            process_ihdr(fh)
        elif(chunk_type == "puNK"):
            key = process_punk(fh, chunk_length)
        else:
            data = fh.read(chunk_length)
            crc = fh.read(4)
    return key

In [12]:
# function that recovers an image encoded into another
def decode_img(image_name,filename,answer):
  # read the image that contains the hidden image
    image = cv2.imread(image_name) #read the image using cv2.imread() 
    if (answer == "no"):
        decoded_img = showData(image)
    elif (answer == "yes"):
        decoded_img = showData_quality(image)
    else:
        print("ERROR: INVALID ANSWER")
    cv2.imwrite(filename, decoded_img)

In [13]:
def decode_text(image_name, path, filename):
  # read the image that contains the hidden image
    image = cv2.imread(image_name) #read the image using cv2.imread() 
    if (len(path) == 0):
        raise ValueError('No keypath was given')
    text = showDataMessage(image, path)
    hidden = ""
    for val in text:
        val = strToInt(val)
        val = chr(val)
        hidden += val
    with open(filename, 'w') as f:
        f.write(hidden)

def decode_text_key(image_name, filename):
  # read the image that contains the hidden image
    key = process_chunks(image_name)
    image = cv2.imread(image_name) #read the image using cv2.imread() 
    text = showDataMessage_key(image, key)
    hidden = ""
    for val in text:
        val = strToInt(val)
        val = chr(val)
        hidden += val
    with open(filename, 'w') as f:
        f.write(hidden)

In [14]:
def win_enc_img3(encode_window):
    def getInput():
        cover = ci_name.get()
        hidden = hi_name.get()
        result = ri_name.get()
        answer = rasp.get()
        encode_img(cover,hidden,result,answer)
        ci_name.delete(0, 'end')
        hi_name.delete(0, 'end')
        ri_name.delete(0, 'end')
        rasp.delete(0, 'end')
    enc_img3 = Toplevel(win)
    enc_img3.geometry("750x250")
    enc_img3.title("Steganosaurus Crypt")
    enc_img3.resizable(width=False, height=False)
    label_mess2=Label(enc_img3, text="Steganosaurus Crypt", font= ('Courier 14 underline'))
    label_mess2.place(relx=0.5, rely=0.07, anchor=CENTER)
    label_mess=Label(enc_img3, text="Encode Image", font= ('Courier 12 underline'))
    label_mess.place(relx=0.5, rely=0.23, anchor=CENTER)
    
    btn_encodeimg = Button(enc_img3, text="Encode Image", command=getInput)
    btn_encodeimg.place(relx=0.75, rely=0.86, anchor=CENTER)
    
    btn_mm = tk.Button(enc_img3, text="Back to Encode Menu", command=lambda:[enc_img3.withdraw(), encode_window.deiconify()])
    btn_mm.place(relx=0.25, rely=0.86, anchor=CENTER)
    cover_image = Label(enc_img3,text="Enter cover image name(with extension): ").place(relx=0.43, rely=0.39, anchor=CENTER)
    ci_name = tk.Entry(enc_img3)
    ci_name.place(relx=0.66, rely=0.39, anchor=CENTER)
    hidden_image = Label(enc_img3,text="Enter hidden image name(with extension): ").place(relx=0.43, rely=0.51, anchor=CENTER)
    hi_name = tk.Entry(enc_img3)
    hi_name.place(relx=0.67, rely=0.51, anchor=CENTER)
    result_image = Label(enc_img3,text="Enter the name of new encoded image(with extension): ").place(relx=0.42, rely=0.63, anchor=CENTER)
    ri_name = tk.Entry(enc_img3)
    ri_name.place(relx=0.7, rely=0.63, anchor=CENTER)
    ans = Label(enc_img3,text="Do you want a better quality image on recover? (yes/no):").place(relx=0.42, rely=0.74, anchor=CENTER)
    rasp = tk.Entry(enc_img3)
    rasp.place(relx=0.71, rely=0.74, anchor=CENTER)
    
def win_dec_img3(decode_window):
    def getInput():
        cover = ci_name.get()
        hidden = hi_name.get()
        answer = rasp.get()
        decode_img(cover,hidden,answer)
        ci_name.delete(0, 'end')
        hi_name.delete(0, 'end')
        rasp.delete(0, 'end')
    dec_img3 = Toplevel(win)
    dec_img3.geometry("750x250")
    dec_img3.title("Steganosaurus Crypt")
    dec_img3.resizable(width=False, height=False)
    label_mess2=Label(dec_img3, text="Steganosaurus Crypt", font= ('Courier 14 underline'))
    label_mess2.place(relx=0.5, rely=0.07, anchor=CENTER)
    label_mess=Label(dec_img3, text="Decode Image", font= ('Courier 12 underline'))
    label_mess.place(relx=0.5, rely=0.23, anchor=CENTER)
    
    btn_encodeimg = Button(dec_img3, text="Decode Image", command=getInput)
    btn_encodeimg.place(relx=0.75, rely=0.86, anchor=CENTER)
    
    btn_mm = tk.Button(dec_img3, text="Back to Decode Menu", command=lambda:[dec_img3.withdraw(), decode_window.deiconify()])
    btn_mm.place(relx=0.25, rely=0.86, anchor=CENTER)
    cover_image = Label(dec_img3,text="Enter the name of the steganographed image that you want to decode (with extension):")
    cover_image.place(relx=0.4, rely=0.39, anchor=CENTER)
    ci_name = tk.Entry(dec_img3)
    ci_name.place(relx=0.8, rely=0.39, anchor=CENTER)
    hidden_image = Label(dec_img3,text="Enter the name of the recovered image(with extension): ")
    hidden_image.place(relx=0.41, rely=0.51, anchor=CENTER)
    hi_name = tk.Entry(dec_img3)
    hi_name.place(relx=0.7, rely=0.51, anchor=CENTER)
    ans = Label(dec_img3,text="Was your image encoded with higher quality? (yes/no):").place(relx=0.42, rely=0.63, anchor=CENTER)
    rasp = tk.Entry(dec_img3)
    rasp.place(relx=0.71, rely=0.63, anchor=CENTER)
    note = Label(dec_img3, text="!!Note: If the image is not what you seek, try changing the answer to the question above!!")
    note.place(relx=0.5, rely=0.74, anchor=CENTER)
    
def win_enc_txt(encode_window):
    def getInput():
        cover = ci_name.get()
        hidden = hi_name.get("1.0",'end-1c')
        answer = rasp.get()
        keyfile = file.get()
        encode_text(cover,hidden,answer, keyfile)
        ci_name.delete(0, 'end')
        hi_name.delete('1.0', END)
        rasp.delete(0, 'end')
        file.delete(0, 'end')
    enc_txt = Toplevel(win)
    enc_txt.geometry("750x250")
    enc_txt.title("Steganosaurus Crypt")
    enc_txt.resizable(width=False, height=False)
    label_mess2=Label(enc_txt, text="Steganosaurus Crypt", font= ('Courier 14 underline'))
    label_mess2.place(relx=0.5, rely=0.07, anchor=CENTER)
    label_mess=Label(enc_txt, text="Encode Text", font= ('Courier 12 underline'))
    label_mess.place(relx=0.5, rely=0.23, anchor=CENTER)
    
    btn_encodeimg = Button(enc_txt, text="Encode Text", command=getInput)
    btn_encodeimg.place(relx=0.75, rely=0.86, anchor=CENTER)
    
    btn_mm = tk.Button(enc_txt, text="Back to Encode Menu", command=lambda:[enc_txt.withdraw(), encode_window.deiconify()])
    btn_mm.place(relx=0.25, rely=0.86, anchor=CENTER)
    cover_image = Label(enc_txt,text="Enter image name(with extension):")
    cover_image.place(relx=0.42, rely=0.39, anchor=CENTER)
    ci_name = tk.Entry(enc_txt)
    ci_name.place(relx=0.63, rely=0.39, anchor=CENTER)
    hidden_image = Label(enc_txt,text="Enter data to be encoded:")
    hidden_image.place(relx=0.42, rely=0.51, anchor=CENTER)
    hi_name = tk.Text(enc_txt, height=1, width=15)
    hi_name.place(relx=0.6, rely=0.51, anchor=CENTER)
    ans = Label(enc_txt,text="Enter the name of new encoded image(with extension):").place(relx=0.43, rely=0.63, anchor=CENTER)
    rasp = tk.Entry(enc_txt)
    rasp.place(relx=0.71, rely=0.63, anchor=CENTER)
    mesaj = Label(enc_txt,text="Enter the name of the key-containing file (add .txt):").place(relx=0.41, rely=0.74, anchor=CENTER)
    file = tk.Entry(enc_txt)
    file.place(relx=0.68, rely=0.74, anchor=CENTER)
    
def win_enc_header(encode_window):
    def getInput():
        cover = ci_name.get()
        hidden = hi_name.get("1.0",'end-1c')
        answer = rasp.get()
        encode_text_key(cover,hidden,answer)
        ci_name.delete(0, 'end')
        hi_name.delete('1.0', END)
        rasp.delete(0, 'end')
    enc_txt = Toplevel(win)
    enc_txt.geometry("750x250")
    enc_txt.title("Steganosaurus Crypt")
    enc_txt.resizable(width=False, height=False)
    label_mess2=Label(enc_txt, text="Steganosaurus Crypt", font= ('Courier 14 underline'))
    label_mess2.place(relx=0.5, rely=0.07, anchor=CENTER)
    label_mess=Label(enc_txt, text="Encode Text and hide key", font= ('Courier 12 underline'))
    label_mess.place(relx=0.5, rely=0.23, anchor=CENTER)
    
    btn_encodeimg = Button(enc_txt, text="Encode Text", command=getInput)
    btn_encodeimg.place(relx=0.75, rely=0.86, anchor=CENTER)
    
    btn_mm = tk.Button(enc_txt, text="Back to Encode Menu", command=lambda:[enc_txt.withdraw(), encode_window.deiconify()])
    btn_mm.place(relx=0.25, rely=0.86, anchor=CENTER)
    cover_image = Label(enc_txt,text="Enter image name(with extension):")
    cover_image.place(relx=0.42, rely=0.39, anchor=CENTER)
    ci_name = tk.Entry(enc_txt)
    ci_name.place(relx=0.63, rely=0.39, anchor=CENTER)
    hidden_image = Label(enc_txt,text="Enter data to be encoded:")
    hidden_image.place(relx=0.42, rely=0.51, anchor=CENTER)
    hi_name = tk.Text(enc_txt, height=1, width=15)
    hi_name.place(relx=0.6, rely=0.51, anchor=CENTER)
    ans = Label(enc_txt,text="Enter the name of new encoded image(with extension):").place(relx=0.43, rely=0.63, anchor=CENTER)
    rasp = tk.Entry(enc_txt)
    rasp.place(relx=0.71, rely=0.63, anchor=CENTER)

def win_dec_txt(decode_window):
    def getInput():
        cover = ci_name.get()
        hidden = hi_name.get()
        answer = rasp.get()
        decode_text(cover,hidden,answer)
        ci_name.delete(0, 'end')
        hi_name.delete(0, 'end')
        rasp.delete(0, 'end')
    dec_txt = Toplevel(win)
    dec_txt.geometry("750x250")
    dec_txt.title("Steganosaurus Crypt")
    dec_txt.resizable(width=False, height=False)
    label_mess2=Label(dec_txt, text="Steganosaurus Crypt", font= ('Courier 14 underline'))
    label_mess2.place(relx=0.5, rely=0.07, anchor=CENTER)
    label_mess=Label(dec_txt, text="Decode Text", font= ('Courier 12 underline'))
    label_mess.place(relx=0.5, rely=0.23, anchor=CENTER)
    
    btn_encodeimg = Button(dec_txt, text="Decode Text", command=getInput)
    btn_encodeimg.place(relx=0.75, rely=0.86, anchor=CENTER)
    
    btn_mm = tk.Button(dec_txt, text="Back to Decode Menu", command=lambda:[dec_txt.withdraw(), decode_window.deiconify()])
    btn_mm.place(relx=0.25, rely=0.86, anchor=CENTER)
    cover_image = Label(dec_txt,text="Enter the name of the steganographed image that you want to decode (with extension) :")
    cover_image.place(relx=0.4, rely=0.39, anchor=CENTER)
    ci_name = tk.Entry(dec_txt)
    ci_name.place(relx=0.8, rely=0.39, anchor=CENTER)
    hidden_image = Label(dec_txt,text="Enter the key path (with extension): ")
    hidden_image.place(relx=0.41, rely=0.51, anchor=CENTER)
    hi_name = tk.Entry(dec_txt)
    hi_name.place(relx=0.62, rely=0.51, anchor=CENTER)
    ans = Label(dec_txt,text="Enter the name of the message-containing file (add .txt):").place(relx=0.4, rely=0.63, anchor=CENTER)
    rasp = tk.Entry(dec_txt)
    rasp.place(relx=0.69, rely=0.63, anchor=CENTER)
    
def win_dec_header(decode_window):
    def getInput():
        cover = ci_name.get()
        answer = rasp.get()
        decode_text_key(cover,answer)
        ci_name.delete(0, 'end')
        rasp.delete(0, 'end')
    dec_txt = Toplevel(win)
    dec_txt.geometry("750x250")
    dec_txt.title("Steganosaurus Crypt")
    dec_txt.resizable(width=False, height=False)
    label_mess2=Label(dec_txt, text="Steganosaurus Crypt", font= ('Courier 14 underline'))
    label_mess2.place(relx=0.5, rely=0.07, anchor=CENTER)
    label_mess=Label(dec_txt, text="Find Key and Decode Text", font= ('Courier 12 underline'))
    label_mess.place(relx=0.5, rely=0.23, anchor=CENTER)
    
    btn_encodeimg = Button(dec_txt, text="Find Key and Decode Text", command=getInput)
    btn_encodeimg.place(relx=0.75, rely=0.86, anchor=CENTER)
    
    btn_mm = tk.Button(dec_txt, text="Back to Decode Menu", command=lambda:[dec_txt.withdraw(), decode_window.deiconify()])
    btn_mm.place(relx=0.25, rely=0.86, anchor=CENTER)
    cover_image = Label(dec_txt,text="Enter the name of the steganographed image that you want to decode (with extension) :")
    cover_image.place(relx=0.4, rely=0.39, anchor=CENTER)
    ci_name = tk.Entry(dec_txt)
    ci_name.place(relx=0.8, rely=0.39, anchor=CENTER)
    ans = Label(dec_txt,text="Enter the name of the message-containing file (add .txt):").place(relx=0.4, rely=0.51, anchor=CENTER)
    rasp = tk.Entry(dec_txt)
    rasp.place(relx=0.69, rely=0.51, anchor=CENTER)
    
def window_encode():
    encode_window = Toplevel(win)
    encode_window.geometry("750x250")
    encode_window.title("Steganosaurus Crypt")
    encode_window.resizable(width=False, height=False)
    
    label_mess2=Label(encode_window, text="Steganosaurus Crypt", font= ('Courier 18 underline'))
    label_mess2.place(relx=0.5, rely=0.13, anchor=CENTER)
    label_mess=Label(encode_window, text="Encode Menu", font= ('Courier 16 underline'))
    label_mess.place(relx=0.5, rely=0.32, anchor=CENTER)

    btn_encodeimg = Button(encode_window, text="Encode Image", command=lambda:[encode_window.withdraw(), win_enc_img3(encode_window)])
    btn_encodeimg.place(relx=0.37, rely=0.72, anchor=CENTER)
    
    btn_encodemes = tk.Button(encode_window, text="Encode Text", command=lambda:[encode_window.withdraw(), win_enc_txt(encode_window)])
    btn_encodemes.place(relx=0.5, rely=0.72, anchor=CENTER)
    
    btn_encodemes = tk.Button(encode_window, text="Encode Text and Key", command=lambda:[encode_window.withdraw(), win_enc_header(encode_window)])
    btn_encodemes.place(relx=0.65, rely=0.72, anchor=CENTER)
    
    btn_mm = tk.Button(encode_window, text="Main Menu", command=lambda:[encode_window.withdraw(), win.deiconify()])
    btn_mm.place(relx=0.5, rely=0.85, anchor=CENTER)
    
    note2 = Label(encode_window, text="IMAGE OUTPUTS MUST NOT HAVE .jpg OR .jpeg EXTENSION")
    note2.place(relx=0.5, rely=0.5, anchor=CENTER)

def window_decode():
    decode_window = Toplevel(win)
    decode_window.geometry("750x250")
    decode_window.title("Steganosaurus Crypt")
    decode_window.resizable(width=False, height=False)
    
    label_mess2=Label(decode_window, text="Steganosaurus Crypt", font= ('Courier 18 underline'))
    label_mess2.place(relx=0.5, rely=0.13, anchor=CENTER)
    label_mess=Label(decode_window, text="Decode Menu", font= ('Courier 16 underline'))
    label_mess.place(relx=0.5, rely=0.32, anchor=CENTER)
    
    btn_decodeimg = Button(decode_window, text="Decode Image", command=lambda:[decode_window.withdraw(), win_dec_img3(decode_window)])
    btn_decodeimg.place(relx=0.37, rely=0.72, anchor=CENTER)
    
    btn_decodemes = tk.Button(decode_window, text="Decode Text", command=lambda:[decode_window.withdraw(), win_dec_txt(decode_window)])
    btn_decodemes.place(relx=0.5, rely=0.72, anchor=CENTER)
    
    btn_decodemes = tk.Button(decode_window, text="Find Key and Decode Text", command=lambda:[decode_window.withdraw(), win_dec_header(decode_window)])
    btn_decodemes.place(relx=0.67, rely=0.72, anchor=CENTER)
    
    btn_mm = tk.Button(decode_window, text="Main Menu", command=lambda:[decode_window.withdraw(), win.deiconify()])
    btn_mm.place(relx=0.5, rely=0.85, anchor=CENTER)
    
    note2 = Label(decode_window, text="IMAGE OUTPUTS MUST NOT HAVE .jpg OR .jpeg EXTENSION")
    note2.place(relx=0.5, rely=0.55, anchor=CENTER)
    
#Create an instance of tkinter frame or window
win=tk.Tk()
#Set the geometry of tkinter frame
win.geometry("750x250")
#Name the window
win.title("Steganosaurus Crypt")

label=Label(win, text="Welcome to Steganosaurus Crypt!", font= ('Courier 20 underline'))
label.place(relx=0.5, rely=0.13, anchor=CENTER)
label2=Label(win, text="Main Menu", font=('Courier 14 underline'))
label2.place(relx=0.5, rely=0.32, anchor=CENTER)

btn_decode = Button(win, text="Decode", command=lambda:[window_decode(), win.withdraw()])
btn_decode.place(relx=0.54, rely=0.8, anchor=CENTER)

btn_encodemes = Button(win, text="Encode", command=lambda:[window_encode(), win.withdraw()])
btn_encodemes.place(relx=0.46, rely=0.8, anchor=CENTER)

frameimg = Frame(win)
frameimg.pack()
frameimg.place(anchor='se', relx=0.95, rely=0.9)

# Create an object of tkinter ImageTk
img = ImageTk.PhotoImage(Image.open("main.png"))

# Create a Label Widget to display the text or Image
labelimg = Label(frameimg, image = img)
labelimg.place(relx=0.95,rely=0.9,anchor='se')
labelimg.pack()

note1 = Label(win, text="ALL INPUTS MUST BE INSIDE APPLICATION FOLDER")
note1.place(relx=0.5, rely = 0.55, anchor=CENTER)
note2 = Label(win, text="OUTPUTS WILL BE PLACED IN THE APPLICATION FOLDER")
note2.place(relx=0.5, rely = 0.65, anchor=CENTER)
win.mainloop()